In [1]:
#install
!pip install python-pptx
!pip install bs4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 5.6 MB/s eta 0:00:00


In [2]:
# import
import requests
from bs4 import BeautifulSoup
from pptx import Presentation
from pptx.util import Inches
from pptx.util import Pt
import smtplib
import os
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders
from email.mime.application import MIMEApplication
import urllib

In [20]:
# 함수 1. 영화리뷰
def review(movie):
  url = f"https://www.rottentomatoes.com/m/{movie}/reviews"
  res = requests.get(url)
  soup = BeautifulSoup(res.text)
  rev = soup.select('p.review-text')
  rev_text = []
  for rev in soup.select('p.review-text'):
    rev_text.append(rev.get_text())
  return rev_text

# 함수 2. 포스터 이미지 저장!!
def movie_img(movie):
  movieR = movie.replace('_',' ')
  keyword = 'movie'+ movieR +'poster'
  url = f"https://www.google.com/search?q={keyword}&source=lnms&tbm=isch&start=100"

  header = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36 Edge/16.16299"}
  html = requests.get(url, headers=header).text

  soup = BeautifulSoup(html, "html.parser")
  img_tag = soup.find_all("img")

  for img in img_tag:
    img_url = img["src"]
    if img_url.startswith("https"):
      img_name = f"{movie}.jpg"
      urllib.request.urlretrieve(img_url, img_name)
      break

# 함수 3. 보고서 자동화
def report(movie, reviews):
  prs = Presentation() # 파워포인트 객체 선언
  title_slide_layout = prs.slide_layouts[0] # 0은 제목슬라이드
  slide = prs.slides.add_slide(title_slide_layout)  # 슬라이드 추가
  title = slide.placeholders[0] # 제목
  title.text = movie # 제목에 값 넣기
  subtitle = slide.placeholders[1]  #부제목
  subtitle.text = "K2024024 정예슬"

  # 빈 슬라이드 생성
  blank_slide_layout = prs.slide_layouts[6]
  slide = prs.slides.add_slide(blank_slide_layout)
  left = Inches(.6)
  top = Inches(.3)
  width = Inches(4.5)
  height = Inches(6)
  revBox = slide.shapes.add_textbox(left, top, width, height)
  tf = revBox.text_frame
  tf.word_wrap = True
  tf.text = ""
  i = 1
  for rev in reviews:
    p = tf.add_paragraph()
    p.text = str(i) + '. ' + rev
    p.level = 0
    p.font.size = Pt(12)
    i+=1
    if i == 11:
      break

  img_path = r'/content/'+movie+'.jpg'
  left = Inches(5.8)
  top = Inches(.8)
  width = Inches(3)
  height = Inches(5)
  imgBox = slide.shapes.add_picture(img_path, left, top, width, height)

  prs.save(movie+'.pptx')

# 함수 4. 메일전송
def send_email(smtp_info,msg):
  with smtplib.SMTP(smtp_info["smtp_server"], smtp_info["smtp_port"]) as server:
    server.starttls()
    server.login(smtp_info["smtp_user_id"], smtp_info["smtp_user_pw"])
    response = server.sendmail(msg['from'],msg['to'],msg.as_string())
    if response:
      print('이메일 전송')
    else:
      print(response)

#main
def main(movie_name, to):
  reviews = []
  file_name = r'/content/'+movie_name+'.pptx'

  reviews = review(movie_name)
  movie_img(movie_name)
  report(movie_name, reviews)

  smtp_info = dict({"smtp_server":"smtp.naver.com","smtp_user_id":"shley0527@naver.com","smtp_user_pw":"yeseul00@","smtp_port":"587"})
  title = '영화 리뷰 입니다.'
  copntent = movie_name + ' 리뷰입니다.'
  sender = smtp_info['smtp_user_id']  #보내는사람 메일계정
  receiver = "shley0527@gmail.com"

  msg = MIMEMultipart()
  msg['Subject'] = title
  msg['From'] = sender
  msg['To'] = receiver
  with open(file_name, 'rb') as ppt_file :
      attachment = MIMEApplication( ppt_file.read() )
      attachment.add_header('Content-Disposition','attachment', filename=movie_name)
      msg.attach(attachment)

  send_email(smtp_info,msg)

In [19]:
to = input('메일을 누구에게 보낼건가요?')
movie = int(input('SELECT MOVIE! 1.deadpool & wolverine | 2.twisters | 3.inside out 2 | 4.despicable me 4 >>> ' ))
if(movie != 1 and movie != 2 and movie != 3 and movie != 4):
  print('다시 입력해주세요)')
elif(movie == 1):
  movie = 'deadpool_and_wolverine'
elif(movie == 2):
  movie = 'twisters'
elif(movie == 3):
  movie = 'inside_out_2'
elif(movie == 4):
  movie = 'despicable_me_4'

main(movie, to)

메일을 누구에게 보낼건가요?a
SELECT MOVIE! 1.deadpool & wolverine | 2.twisters | 3.inside out 2 | 4.despicable me 4 >>> 4


발표 ppt내용

1. Get movie reviews by e-mail (영화 리뷰를 메일로 받기) -- K2024024
2. api 작동순서 (메일 수신 계정과 영화 제목 입력 -> roton tomato에서 영화 리뷰 크롤링 -> ppt 작성 -> 메일전송)
3. 코드구성(함수부분 & 호출부분)
4. 결과물 화면
5. end!